In [ ]:
### TF*IDF 원시적 방법

# 불러오기
from math import log # idf연산에 필요한 로그
import pandas as pd 
t_data = [ 
    '공부 하고 싶다' ,
    '공부 하고 싫다' ,
    '공부 내일 할까' ,
    '공부 하고 놀기' ,
    '놀기 말고 놀자' ] 

# 단어 (보캐불러리셋집합만들기)
vos = list(set( 단어 for 문장 in t_data for 단어 in 문장.split())) # 중복방지를위한set # t_data에서 문장을 꺼내서 스플릿해서 단어를 꺼내서 리스트로 만든다.

# TF 특정문서에서 특정 단어의 등장 횟수 (DTM의 값)
# DT 특정 단어가 등장한 문장의 수
# IDF 디티에 반비례하는 수 (로그 1플러스 디티 분의 엔)
# TFIDF : TF 곱하기 IDF , DTM 데이터프레임에 각 밸류에 각각의 가중치를 곱한 것

# TF-IDF  공식 코드화

# TF 빈도
n=len(t_data) # 문장의 개수
def tf(t,d):       
    return d.count(t)    # d 문서에 t 단어가 몇번 나왔는지 count 

# IDF 가중치, 역빈도(문장안의 빈도가 적으면 오히려 주요단어일 가능성이 높아서 가중치가 높다)
def idf(t):
    df=0 
    for 문장 in t_data:
        df +=  t in 문장   # 문장안에 t 가 있다면 1이 df에 전달됨 > df 에 +1 씩 증가함 
    return log(n/(df+1))                   # idf 공식    idf(d,t) = log ( n / 1+df(t) )

# TFIDF
def tfidf(t,d):
    return tf(t,d)*idf(t)

# DTM = tf table
t_l = []
for i in range(n): # 문장개수
    t_l.append([]) # 이차원리스트를만들기위한 [ [],[],[] ] #인덱스넣을칸작업
    d = t_data[i] # 문서에서 문장을 하나 뽑기
    for j in range(len(vos)): #vos 보캐불러셋집합
        t = vos[j] # 보캐에서 단어 하나를 뽑고
        t_l[-1].append(tf(t,d))   # [ [] ] 에 단어의 빈도를 넣는다 
import pandas as pd
tf_ = pd.DataFrame(t_l, columns = vos)
tf_  # 싶다 - 1이 하나뿐 > 문장에서 주요의미 > 가중치 높게

# IDF 연산 후 데이터프레임으로 만들기
idf_l = []
for i in range(len(vos)):
    t = vos[i] # 보캐에서 단어 하나를 뽑음
    idf_l.append(idf(t)) # IDF(t)로 단어t의 가중치를 뽑아서 리스트에 넣음
idf_ = pd.DataFrame(idf_l , index=vos , columns=['IDF']) 
idf_ # 각 보캐단어별 idf  가중치를 보여준다  (단어마다 weight 곱셈 = 벡터화)

# TFIDF를 연산 후 데이터프레임으로 만들기
tf_idf_l = []
for i in range(n): # n: 문장 개수
    tf_idf_l.append([]) # [ [] ] 이차원리스트생성
    d = t_data[i]  # 데이터에서 문서하나를 뽑음
    for j in range(len(vos)):  # 보캐len만큼
        t = vos[j] # 보캐에서 단어 하나씩 뽑음
        tf_idf_l[-1].append(tfidf(t,d)) # 이차원빈리스트에 TFIDF 값을 넣음
tf_idf_ = pd.DataFrame(tf_idf_l, columns=vos )
tf_idf_  # 데이터테이블: tf 와 idf 곱   tf-idf

# 국소표현.이산표현  BOW DTM  TF-IDF  (임베딩도 일종의 국소표현 임베딩을 하면 된다 단 유사도검사시 TF IDF로)
# 분산표현.연속표현  Word2Vec, FastText

# TFIDF 전용 라이브러리가 있기 때문에 위 코드는 더 쓰진 않는다.

In [ ]:
### TF*IDF 라이브러리 활용

# TF IDF 벡터라이저 불러오기
from sklearn.feature_extraction.text import TfidfVectorizer # TF IDF 값을 정규화한 형태
t_data = [ 
    '공부 하고 싶다' ,
    '공부 하고 싫다' ,
    '공부 내일 할까' ,
    '공부 하고 놀기' ,
    '놀기 말고 놀자' ] 
tfidfv = TfidfVectorizer().fit(t_data)
tfidfv.vocabulary_ # 단어:인덱스 의 t_data 기준 보캐불러리 생성
a = tfidfv.transform(t_data).toarray() # TF IDF 어레이
pd.DataFrame(a, columns = ['공부','내일','놀기','놀자','말고','싫다','싶다','하고','할까']) # TF IDF 데이터프레임으로 # tfidfv.vocabulary_ 로 칼럼확인